# Projet immo IA / partie EDA

# Import librairie

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# Import fichier

In [3]:
df = pd.read_csv("train_data.csv")
df = df.rename(columns =  {"Unnamed: 0" : "id"})
df

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.csv'

# 1ère itération

## Visualisation

On tente de visualiser toute nos variables les unes par rapport aux autres pour en tirer des informations si possible

Ici on peux voir des corrélation positive entre total_bedrooms, total_rooms, population, median_income,lattitude et longitude

In [ ]:
sns.pairplot(df, corner = True, hue = "ocean_proximity")
plt.show()

Ici on peut voir la répartition du prix des maisons en fonction de leur proximité avec l'ocean. On remarque très clairement que les maisons les plus chers se situe en bord d'océan

In [ ]:
fig = px.scatter_mapbox(df, lat="latitude", 
lon="longitude",color="median_house_value", mapbox_style= "open-street-map",zoom = 4.5)
fig.update_layout(
    showlegend=False,
    font={'size': 15},
    title={'text': '<b>Median</b>', 'font': {'size': 20}}, 
    title_x = 0.45
)

fig.show()

Sans surprise les ménage les plus pauvre sont éloigné de l'ocean où les maisons coutent le plus cher. 
Ce qui nous donne une indication sur la corélation entre ces trois variables

In [ ]:
sns.scatterplot(data = df, x= "median_income", y = "median_house_value", hue = "ocean_proximity")
plt.show()

## Duplicates

On vérifie que chaque id sois unique, si tel est le cas comme ici, on supprime la colonne pour tester si nous n'avons pas un bloc qui aurait été crée en double 

In [ ]:
df["id"].unique

In [ ]:
df.drop(columns = "id").drop_duplicates()

## Missing data

On va vérifier le nombre de valeurs manquantes dans notre dataset en regardant les NAN value ou bien les valeurs qui serait à 0

Ici nous avons 176 NaN value sur total_bedrooms et toute nos données sur le reste

In [ ]:
df.isnull().sum()

Ensuite on vérifie le nombre de 0 présent dans le dataset
ici on à juste l'id à 0 

In [ ]:
zero = (df==0).any().sum(axis = 0)
print("Number of zero :", zero)
df.loc[df["id"]==0]

In [ ]:
df.loc[df["total_bedrooms"].isna()]

## Outliers

Nous recherchons les Outliers pour peut être identifier ce qui sont et ensuite les supprimé.
Notre modèle de ML peut être fortement biaisé par les outliers donc nous les supprimeront

Initialement en l'état on ne peux pas regarder les outliers en ayant tout les boxplot sur un graphique.
Pour palier à ce problème nous allons utiliser MinMaxScaler de scikit learn pour standardiser nos point entre 0 et 1

In [ ]:
plt.subplots(figsize = (20,10))
sns.boxplot(data = df)
plt.show()

Comme on peux le voir, en ayant nos features entre 0 et 1 le graphique est bien plus clair

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_numerique = df.copy().drop(columns = ["ocean_proximity","id"])
scaler = MinMaxScaler()
arr_scale = scaler.fit_transform(df_numerique)
df_scale = pd.DataFrame(arr_scale, columns = df_numerique.columns)
plt.subplots(figsize = (20,10))
sns.boxplot(data = df_scale)
plt.show()

In [ ]:
df_scale = df_scale.fillna(0)
df_without_outliers = df_scale[(np.abs(stats.zscore(df_scale["total_rooms"])) < 2)& 
         (np.abs(stats.zscore(df_scale["total_bedrooms"])) < 2) & 
         (np.abs(stats.zscore(df_scale["population"])) < 2) &
         (np.abs(stats.zscore(df_scale["households"])) < 2) &  
         (np.abs(stats.zscore(df_scale["median_income"])) < 2) &
         (np.abs(stats.zscore(df_scale["median_income"])) < 2)]
plt.subplots(figsize = (20,10))
sns.boxplot(data = df_without_outliers)
plt.show()

In [ ]:
Pour supprimez les outliers on utilise le zscore, et on prendra toute les valeurs ayant un zscore inférieur à 2.
2 étant la limite qui représente 95% de nos données.
Si on plaçait cette limite à 3 on aurait 99.7% de nos données, ce qui ne serait pas suffisant pour supprimez assez d'outlier